In [8]:
import matplotlib.pyplot as plt
try:
    xrange = xrange
except:
    xrange = range
from PIL import ImageGrab
import cv2
import time
import math
import datetime
import random
import io
from lib.getkeys import key_check
from lib.reinforcement import Qnetwork,updateTarget,updateTargetGraph
from lib.SQL import SQLCalls
from sys import stdout
import sqlite3
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import os
from PIL import Image
from keras import backend as K
from keras.utils import plot_model
from keras.models import Model,Sequential
from keras.layers import Input, LSTM, Dense, Dropout, Conv2D, MaxPooling2D,concatenate, Flatten, GlobalAveragePooling2D
from keras.utils import to_categorical
SQL=SQLCalls()

In [ ]:
from PIL import Image
def process_img(original_image):
    processed_img= cv2.resize(original_image,(580,580))
    return np.array(np.transpose(processed_img,(0,1,2)))
print_screen = np.array(ImageGrab.grab(bbox=(0,60,580,530)))
print(print_screen.shape)
x=process_img(print_screen)
Image.fromarray(x,'RGB')

In [ ]:

epoch=0
frame_count=0
ACTION,WAIT,DEATH,GENERATION_OVER,RESTORE=0,1,2,3,4
print("Taking picture of the top-left of the screen.")
print("Please check image to ensure it only displays the emulator.")
img=ImageGrab.grab(bbox=(0,60,580,530))
img.save("../Test.png")

#Hyper Params
update_freq = 4 #How often to perform a training step.
y = .1 #Discount factor on the target Q-values
startE = 1 #Starting chance of random action
endE = 0.1 #Final chance of random action
anneling_steps = 10000. #How many steps of training to reduce startE to endE.
pre_train_steps = 10000 #How many steps of random actions before training begins.
max_epLength = 50 #The max allowed length of our episode.
load_model = False #Whether to load a saved model.
path = "./dqn" #The path to save our model to.
h_size = 1024 #The size of the final convolutional layer before splitting it into Advantage and Value streams.
tau = 0.001 #Rate to update target network toward primary network
img_size=84 #Size of the image.

In [69]:
cur.execute("SELECT GenomeNum,Gene,GeneContent FROM Genes ORDER BY Genome,Gene")
GenomesUnformatted=cur.fetchall()
print(GenomesUnformatted[1])

(1, 2, '258 1000004 1.4357737968078')


In [196]:
Genomes=SQL.GatherGenomes()
POPULATION=len(Genomes)
#print(POPULATION)
#print(Genomes[0])
batch_size = POPULATION//4 #How many experiences to use for each training step.
BoxRadius=6
BoxLength=BoxRadius*2+1
BoxArea=(BoxLength)*(BoxLength)
gene_image=np.empty([len(Genomes),BoxLength,BoxLength,12])
gene_image.fill(0)
BUTTON_AMOUNT=6
print(Genomes[82])
for Genome_Num,Genome in enumerate(Genomes):
    if Genome_Num==0:
        print(Genome)
        for gene in Genome:
            genome_type=0
            if gene[0]<BoxArea:
                pass
                #Normal Input")
            elif gene[0]>BoxArea*2:
                continue
                #bias
            else:
                pass
                #print("Inverse Input")
                genome_type+=BUTTON_AMOUNT
            genome_type+=int(gene[1]-1000001) 
            if genome_type>=0:
                # print X,Y,Type(Type of Input,Button Pressed)
                gene_image[Genome_Num][int(gene[0]%(BoxArea)//BoxLength)][int(gene[0]%(BoxArea)%13)][genome_type]=gene[2]             
'''
[[    339 1000003       9]
 [    258 1000004       7]
 [    178 1000004       8]
'''

for l in range(6):
    for i in range(len(gene_image[l])):
        #print(gene_image[l][i])
        for j in range(len(gene_image[l][i])):
            for k in range(len(gene_image[l][i][j])):
                if gene_image[l][i][j][k]!=0:
                    print(l,i,j,k)
                    print(gene_image[l][i][j][k])
         

[[205.0, 1000003.0, -1.933835871456]]
[[339.0, 1000003.0, -1.3364055299539], [258.0, 1000004.0, 1.4357737968078], [178.0, 1000004.0, -1.9400616473891]]
0 0 9 9
-1.94006164739
0 6 11 9
1.43577379681


In [10]:
def adapt_array(arr):
    """
    http://stackoverflow.com/a/31312102/190597 (SoulNibbler)
    """
    out = io.BytesIO()
    np.save(out, arr)
    out.seek(0)
    return sqlite3.Binary(out.read())
def convert_array(text):
    out = io.BytesIO(text)
    out.seek(0)
    return np.load(out)

In [3]:
def setup_genomes():
    BoxRadius=6
    BoxLength=BoxRadius*2+1
    BoxArea=(BoxLength)
    gene_image=np.empty([len(Genomes),BoxLength,BoxLength,12])
    gene_image.fill(0)
    BUTTON_AMOUNT=6
    for Genome_Num,Genome in enumerate(Genomes):
        for gene in Genome:
            genome_type=0
            if gene[0]>BoxLength:
                pass
                #print("Normal Input")
            if gene[1]>BoxLength:
                pass
                #print("Inverse Input")
                genome_type+=BUTTON_AMOUNT
            genome_type+=int(gene[1]-1000001) 
            if genome_type>=0:
                # print X,Y,Type(Type of Input,Button Pressed)
                gene_image[Genome_Num][int(gene[0]%(BoxArea)//BoxArea)][int(gene[0]%(BoxArea)%13)][genome_type]=gene[2] 
    return gene_image

In [11]:
sqlite3.register_adapter(np.ndarray, adapt_array)

# Converts TEXT to np.array when selecting
sqlite3.register_converter("array", convert_array)

con=sqlite3.connect('DQN.db', detect_types=sqlite3.PARSE_DECLTYPES,isolation_level=None)
cur = con.cursor()
cur.execute("PRAGMA synchronous = OFF;")
cur.execute("PRAGMA journal_mode=WAL;")
cur.execute("PRAGMA read_uncommitted = true;")
cur.execute("SELECT GenomeNum,Gene,GeneContent FROM Genes ORDER BY Genome,Gene")
# trainBatch=SQL.gain_history()
# batch_size=20
# trainBatch=trainBatch[0:batch_size]
# time_stamp=datetime.datetime.now().time()
# genomeImages=setup_genomes()
# genomeImages=genomeImages[0:batch_size]
# for i in range(len(genomeImages)):
#     sql = ''' INSERT INTO example_genes
#                (GenomeKey,GeneImage)
#                VALUES
#                (?,?)'''
#     cur.execute(sql, (str(time_stamp)+str(i),genomeImages[i]))
# con.commit()    
# [0] Image
# [1] GenomeNUm
# [2] Score
# [3] ImageEn d
# states=trainBatch[:,0]
# GenomeNum=trainBatch[:,1]
# score=trainBatch[:,2]
# states_after=trainBatch[:,3]
# for i in range(len(trainBatch)):
#     sql=''' INSERT INTO example_images
#                (GenomeKey,Score,Image,ImageEnd)
#                VALUES
#                (?,?,?,?)'''
#     cur.execute(sql, (states[i],GenomeNum[i],score[i],states_after[i]))
# con.commit()  
# sql=''' INSERT INTO example_timestamps
#                 (timestamp)
#                 VALUES
#                 (?)'''
# cur.execute(sql, (str(time_stamp),))
# con.commit()    


In [12]:
sql = '''Select image,score,imageEnd,geneImage,ei.GenomeKey,eg.GenomeKey
        from example_images ei
        inner join 
        example_genes eg on ei.genomeKey==eg.genomeKey
        where score=-1 LIMIT 99'''
cur.execute(sql)
results=cur.fetchall()
results=np.array(results)
np.random.shuffle(results)
sql = '''Select image,score,imageEnd,geneImage, ei.GenomeKey,eg.GenomeKey 
        from example_images ei
        inner join 
        example_genes eg on ei.genomeKey==eg.genomeKey
        where score=0 
        LIMIT 99'''
cur.execute(sql)
results2=cur.fetchall()
results2=np.array(results2)
np.random.shuffle(results2)
# sql = '''Select image
#         from rewards
#         where GenomeNum=1 LIMIT 5'''
# cur.execute(sql)
# results3=cur.fetchall()
# results3=np.array(results3)
# np.random.shuffle(results3)

20


In [13]:
def make_model():
    image_model_inputs = Input(shape=X[0].shape,dtype='float32',name='main_image')
    image_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(image_model_inputs)
    image_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(image_model)

    image_model=Conv2D(32, (1,1),strides=2, padding='valid', activation='relu')(image_model)
    image_model=Conv2D(32, (1,1),strides=2, padding='valid', activation='relu')(image_model)

    image_model=Conv2D(8, (1,1),strides=3, padding='same', activation='relu')(image_model)
    image_model=Conv2D(8, (1,1),strides=2, padding='same', activation='relu')(image_model)
    
    image_model=Flatten()(image_model)
    
    gene_model_inputs = Input(shape=X_gene[0].shape,dtype='float32',name='gene_image')
    gene_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(gene_model_inputs)
    gene_model=Conv2D(16, (2, 2), padding='valid', activation='relu')(gene_model)
    
   # gene_model=Conv2D(8, (1,1),strides=3, padding='same', activation='relu')(gene_model)
    #gene_model=Conv2D(8, (1,1),strides=2, padding='same', activation='relu')(gene_model)
    
    gene_model=Flatten()(gene_model)
    
    combined_model=concatenate([image_model,gene_model])
    
    combined_model=Dense(32, activation='relu')(combined_model)
    combined_model=Dense(8, activation='relu')(combined_model)
    combined_model_preditions=Dense(2, activation='softmax')(combined_model)
    model=Model(inputs=[image_model_inputs,gene_model_inputs],outputs=combined_model_preditions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
K.clear_session()
X=np.reshape(np.vstack(np.concatenate((results[:,0],results2[:,0]))),(-1,580,580,3))
X_gene=np.reshape(np.vstack(np.concatenate((results[:,3],results2[:,3]))),(-1,13,13,12))
Y=np.vstack(np.concatenate((results[:,1],results2[:,1])))
Y = to_categorical(Y, num_classes=2)
train=170
y_train=Y[:train]
y_test=Y[train:]
x_train=X[:train]
x_test=X[train:]
x_gene_train=X_gene[:train]
x_gene_test=X_gene[train:]
model = make_model()
plot_model(model, to_file='multilayer_perceptron_graph.png',show_shapes=True)

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_image (InputLayer)          (None, 580, 580, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 579, 579, 16)  208         main_image[0][0]                 
____________________________________________________________________________________________________
conv2d_2 (Conv2D)                (None, 578, 578, 16)  1040        conv2d_1[0][0]                   
____________________________________________________________________________________________________
conv2d_3 (Conv2D)                (None, 289, 289, 32)  544         conv2d_2[0][0]                   
___________________________________________________________________________________________

In [217]:
print(results2[:,4][0])
print(results2[:,4][1])
#Image.fromarray(results2[:,0][0],'RGB')
Image.fromarray(results3[:,0][0],'RGB')
gene=results2[:,3][0]
#gene=gene_image[52]
for row in gene:
    for col in row:
        #print(col)
        for button in col:
            pass
            if button!=0.0: 
                print("{0:.2f}".format(button),end=' ')
       # print(" ")    
    #print(" ")        

05:54:39.08542826
05:43:18.1571557
2.16 -1.47 -2.03 0.59 0.96 -1.17 -0.47 1.63 1.10 -0.56 1.09 -1.14 -1.98 1.29 -2.02 1.70 0.82 -0.05 -1.98 0.34 1.58 1.09 0.80 1.12 1.75 -0.29 1.56 

In [6]:
epochs=40
batch_size=16
history = model.fit([x_train,x_gene_train], y_train, batch_size=batch_size, epochs=epochs, verbose=1,validation_data=([x_test,x_gene_test], y_test))

#model1.evaluate(test_data, test_labels_one_hot)

Train on 170 samples, validate on 28 samples
Epoch 1/40
170/170 [==============================] - 6s - loss: 4.9492 - acc: 0.5765 - val_loss: 13.3361 - val_acc: 0.1429
Epoch 2/40
170/170 [==============================] - 3s - loss: 2.6320 - acc: 0.6824 - val_loss: 0.1272 - val_acc: 0.9643
Epoch 3/40
170/170 [==============================] - 3s - loss: 0.7294 - acc: 0.6118 - val_loss: 0.1178 - val_acc: 0.9643
Epoch 4/40
170/170 [==============================] - 3s - loss: 0.5043 - acc: 0.7647 - val_loss: 0.4988 - val_acc: 0.7857
Epoch 5/40
170/170 [==============================] - 3s - loss: 0.3469 - acc: 0.8706 - val_loss: 0.8295 - val_acc: 0.5357
Epoch 6/40
170/170 [==============================] - 3s - loss: 0.2999 - acc: 0.8941 - val_loss: 0.6489 - val_acc: 0.5714
Epoch 7/40
170/170 [==============================] - 3s - loss: 0.2299 - acc: 0.9118 - val_loss: 0.2191 - val_acc: 0.9286
Epoch 8/40
170/170 [==============================] - 3s - loss: 0.1528 - acc: 0.9588 - val_l

In [24]:
from keras.models import load_model
#model = load_model('dqn_frozen_model.h5')
image_duplicated=np.tile(x_gene_test[0], (20,1,1,1))
print(x_test.shape)
print(x_gene_test.shape)
# history = model.predict([x_test,image_duplicated])


(28, 580, 580, 3)
(28, 13, 13, 12)


In [21]:
print(history)
print(len(history))

[[  1.00000000e+00   5.09079250e-08]
 [  9.98849392e-01   1.15060795e-03]
 [  9.99998212e-01   1.76865592e-06]
 [  9.99912143e-01   8.78875144e-05]
 [  9.99999404e-01   5.77715070e-07]
 [  9.99923110e-01   7.69332109e-05]
 [  9.99933004e-01   6.70404406e-05]
 [  9.99997854e-01   2.18971627e-06]
 [  1.00000000e+00   9.48082538e-15]
 [  9.99987364e-01   1.25802708e-05]
 [  9.99975920e-01   2.40807603e-05]
 [  9.98849392e-01   1.15060795e-03]
 [  1.00000000e+00   7.61649677e-09]
 [  9.99933004e-01   6.70404406e-05]
 [  9.99913812e-01   8.61948211e-05]
 [  9.99995112e-01   4.83344411e-06]
 [  9.99981761e-01   1.82758449e-05]
 [  9.99943733e-01   5.62598652e-05]
 [  9.99985456e-01   1.45408703e-05]
 [  9.99992847e-01   7.13469080e-06]
 [  1.00000000e+00   2.73720716e-13]
 [  1.00000000e+00   6.67478156e-11]
 [  9.99999642e-01   3.01593644e-07]
 [  9.99989748e-01   1.02680660e-05]
 [  1.00000000e+00   7.34864155e-13]
 [  9.99899268e-01   1.00769241e-04]
 [  9.99999523e-01   4.59988485e-07]
 

In [7]:
model.save('dqn_frozen_model.h5')

In [224]:
saver = tf.train.Saver()
path = "./dqnnew"
epoch=40
saver.save(K.get_session(),path+'/model-'+str(epoch)+'.ckpt',global_step=epoch)

'./dqnnew/model-40.ckpt-40'

In [39]:
def make_model():
    model = Sequential()
    model.add(Conv2D(16, (2, 2), padding='valid', activation='relu', input_shape=X[0].shape))
    model.add(Conv2D(16, (2, 2), padding='valid', activation='relu'))
    #model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Conv2D(8, (1, 1),strides=3, padding='same', activation='relu'))
    model.add(Conv2D(8, (1, 1),strides=2, activation='relu'))
    #model.add(MaxPooling2D(pool_size=(3, 3)))

    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model
K.clear_session()
X=np.reshape(np.vstack(np.concatenate((results[:,3],results2[:,3]))),(-1,13,13,12))
np.random.shuffle(X)
Y=np.vstack(np.concatenate((results[:,1],results2[:,1])))
Y = to_categorical(Y, num_classes=2)
np.random.shuffle(Y)
y_train=Y[0:130]
y_test=Y[130:150]
x_train=X[0:130]
x_test=X[130:150]
model = make_model()
plot_model(model, to_file='multilayer_perceptron_graph.png')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 12, 12, 16)        784       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 16)        1040      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 8)           136       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 2, 8)           72        
_________________________________________________________________
flatten_1 (Flatten)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
__________

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.